In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Carpeta del dataset
dataset_folder = "dataset"

# Lista para almacenar los datos
data = []

# Diccionario de nombres de los puntos de referencia en MediaPipe
landmark_names = mp_pose.PoseLandmark

# Recorrer cada subcarpeta que representa una categoría de pose
for pose_folder in os.listdir(dataset_folder):
    pose_path = os.path.join(dataset_folder, pose_folder)

    if os.path.isdir(pose_path):
        images = [os.path.join(pose_path, img) for img in os.listdir(pose_path) if img.endswith(('.jpg', '.png'))]

        for img_path in images:
            image = cv2.imread(img_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if not results.pose_landmarks:
                continue  # Si no hay landmarks, saltar la imagen

            # Obtener coordenadas de los landmarks
            landmarks = {lm_idx: (lm.x, lm.y, lm.z) for lm_idx, lm in enumerate(results.pose_landmarks.landmark)}

            # Generar los vectores de cada arista
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                
                # Obtener los nombres de los puntos de conexión
                start_name = landmark_names(start_idx).name
                end_name = landmark_names(end_idx).name
                
                # Crear nombre del vector
                vector_name = f"{start_name}_{end_name}"

                # Obtener las coordenadas
                x1, y1, z1 = landmarks[start_idx]
                x2, y2, z2 = landmarks[end_idx]

                # Calcular el vector
                vec_x = x2 - x1
                vec_y = y2 - y1
                vec_z = z2 - z1

                # Agregar los datos
                data.append([vector_name, vec_x, vec_y, vec_z, pose_folder])

# Cerrar MediaPipe Pose
pose.close()

# Crear un DataFrame con los datos
df = pd.DataFrame(data, columns=["name", "x", "y", "z", "classification"])

# Guardar en un archivo CSV
df.to_csv("knn.csv", index=False)

print("Archivo CSV generado con éxito.")


Archivo CSV generado con éxito.
